In [1]:
from __future__ import division
import tensor_calculation as t
import tensor_advanced as ta
from collections import Counter
import random
from math import exp

In [2]:
def logist(list_):
    if type(list_) != list:
        return (1/(1+exp(-1*list_)))
    return [logist(i) for i in list_ ]     

In [3]:
def de_logist(list_):
    return t.list_muti(logist(list_), t.list_shift(t.list_nu(logist(list_), -1) , 1))

In [4]:
def tanh(list_):
    if type(list_) != list:
        return (1-exp(-2*list_/100))/(1+exp(-2*list_/100))
    return [tanh(i) for i in list_ ] 

In [5]:
def de_tanh(list_):
    inter = t.list_power(tanh(list_), 2)
    return t.list_shift(t.list_nu(inter, -1) ,1)

In [6]:
def relu(list_):
    if type(list_) != list:
        return max(list_,0 )
    return [relu(i) for i in list_ ] 

In [7]:
def de_relu(list_):
    if type(list_) != list:
        if list_>0:
            return 1
        elif list_==0:
            return 1/2
        else:
            return 0
    return [de_relu(i) for i in list_ ] 

In [8]:
def zero_return1(list_):
    if type(list_) != list:
        if list_>0:
            return list_
        return 1
    return [zero_returrn1(i) for i in list_ ] 

In [9]:
def actvi_func(list_, (i, in_list)): #to compute  propagation and back propagation using the same function
    if i>0:
        if i==1:
            return logist(list_)
        elif i ==2:
            return tanh(list_)
        elif i ==3:
            return relu(list_)
        elif i ==4:
            return ta.softmax(list_)
    elif i<0:
        if i==-1:
            return t.list_muti(de_logist(in_list), list_)
        elif i ==-2:
            return t.list_muti(de_tanh(in_list), list_)
        elif i ==-3:
            return t.list_muti(de_relu(in_list), list_)
        elif i ==-4:
            return ta.mat_dot(list_, ta.de_softmax(in_list) )
    else:         #do nothing if i =0
        return list_

In [10]:
def revers(list_):
    if type(list_) != list:
        return list_
    return [ list_[len(list_)-i-1] for i in range(len(list_)) ]

In [11]:
def dnn (x , w, b, func, in_list): # to give propagation and back propagation the same treatment
    if b == []:
        b = [ [] for k in range(len(w))]
    if len(w) != len(b) or len(x) !=len(w[0][0]):
        return "wrong parameter"
    if type(func) == int:
        if func >0:
            func_list=[(0,0)]+[(func, 0)]*len(b)
        else:
            func_list = [(func, revers(in_list)[i]) for i in range(len(b))]+[(0,0)]
    elif len(func) == len(b):
        if func[0] >0:
            func_list = [(0,0)] + [(func[k], 0)  for k in  range(len(func))]
        elif func[0] <0:
            func_list = [(func[k], revers(in_list)[k]) for k in  range(len(func))]+[(0,0)]
    elif len(func) == len(b)+1:
        func_list = func
    else:
        return "wrong func"
    o_tensor, i_tensor=[], [x]
    for i in range(len(func_list)): 
        o_tensor.append(actvi_func(i_tensor[i], func_list[i]) )
        if i == len(w):
            break
        i_tensor.append(t.list_add(ta.mat_dot(w[i], o_tensor[i]), b[i]))
    return [o_tensor, i_tensor]

In [12]:
www=ta.gene_random_tensor([1, 3, 2] ,(0,1))+ta.gene_random_tensor([1, 4, 3] ,(0,1))\
 +ta.gene_random_tensor([1, 1, 4] ,(0,1))
bbb=ta.gene_random_tensor([1, 3] ,(0,1) )+ ta.gene_random_tensor([1, 4] ,(0,1) )\
+ta.gene_random_tensor([1, 1] ,(0,1) )

In [13]:
dnn ([1,0] ,www, bbb, 1, [])[0]

[[1, 0],
 [0.7633030879610497, 0.6609117494945463, 0.6582476056887406],
 [0.8309924509219907,
  0.8371753606021455,
  0.7623174230949795,
  0.7766593264443463],
 [0.9170995303422443]]

In [14]:
def der_sqloss_out(y, predict): #using square loss
    return t.list_sub(predict, y)

In [15]:
def der_kvloss_out(y, predict):
    return [-yy/p for (yy, p) in zip(y, predict) ]

In [16]:
def derw_b_pre(data_instance, w, b, func): 
    bw = [ta.mat_transp(k) for k in revers(w)] 
    new_func = revers(t.list_nu(func, -1))
    out_tensor = dnn(data_instance[0] ,w, b, func, [])[0]
    in_tensor = dnn(data_instance[0] ,w, b, func, [])[1]
    predict = out_tensor[-1:][0]
    #print in_tensor[-1:][0]
    der_loss_out=der_kvloss_out(data_instance[1], predict)
    delta_list= revers(dnn(der_loss_out, bw, [], new_func, in_tensor)[0])
    return [[ta.make_mat(u,v) for (u,v) in zip(delta_list[1:],out_tensor[:-1])] ,delta_list[1:], predict]

In [17]:
t.list_add(derw_b_pre([[1,2],[1]], www, bbb, 1)[0] ,www)

[[[0.5195399176140828, 0.9348595543156384],
  [0.3524463154069381, 0.06588081356012682],
  [0.6267204156465814, 0.523383679845041]],
 [[0.4560966164802988, 0.2614524674550623, 0.803169934406284],
  [0.28606854404372645, 0.19378546714260728, 0.642383752792055],
  [0.20520346399662506, 0.9135497741353451, 0.27259038936645763],
  [0.021139804897665534, 0.9331354014232832, 0.17919688675703646]],
 [[0.7701892045832053,
   0.7272475470273453,
   0.35124888391144593,
   0.05972466616633936]]]

In [74]:
def train_dnn(data, struct, func, step, ini):
    w = [ta.gene_random_tensor([ i, j] ,(-abs(ini), abs(ini)) ) for (i,j) in  \
            zip(struct+ [len(data[0][1])], [len(data[0][0])]+struct ) ]
    b = []
    n=1
    mini = 50
    #print w
    #batch = random.sample(data, 100)
    while True:
        error= []
        batch = random.sample(data, 100)
        sum_e_w=[]
        sum_e_b=[]
        for d in batch:
            [der_w, der_b, predict] = derw_b_pre(d, w, b, func)
            error += [ta.kl_div(d[1], predict)]
            error.append(t.list_contrac(t.list_sub( predict , d[1]), t.list_sub( predict , d[1])))
            sum_e_w.append(der_w)
            sum_e_b.append(der_b)
        print sum(error)
        wder_e_w=t.list_sum(sum_e_w)
        wder_e_b=t.list_sum(sum_e_b)
        w = t.list_add(w, t.list_nu(wder_e_w, -1*step/(n)**(1/2)))
        b = t.list_add(b, t.list_nu(wder_e_b, -1*step/(n)**(1/2)))
        n +=1
        if mini >sum(error):
            mini = sum(error)
        #print mini
        if  n==1000: #or mini<0.3:
            break
    return [w, b]      

In [19]:
data_x=ta.gene_random_tensor([1000,2], (-10, 10))
data1 = [[x, [1,0] ]  if x[0]**2+ x[1]**2<7**2 else [x, [0,1] ] for x in data_x ]
data2 = [[x, [1,0] ]  if x[0]+ x[1]<0 else [x, [0,1] ] for x in data_x ]
data3 =[ [x,x] for x in data_x]

In [20]:
data_n=ta.gene_random_tensor([100,2,1], (-1, 1)), [[1/2,1/2],[1,0]], [[1/2,1/2],[1,0]], [[1/2,1/2],[1,0]], [[1/2,1/2],[1,0]]

In [21]:
data=[ [[0,1],[0,1]], [[1,1],[1,1]]]

In [22]:
for i in range(10):
    data.append([ta.gene_random_tensor([1,2],(-1,1))[0], ta.gene_random_tensor([1,2],(-1,1))[0]])

In [75]:
train_dnn(data1, [2,2,2], [3,3,3,4], .1, .1)

119.314900624
157.39419465
160.558073243
144.015989885
115.323291907
111.007211651
111.691461219
107.688360427
104.771865628
115.992296989
116.072202843
118.283464675
114.942788134
104.874819032
104.69487972
114.033233623
112.538736012
104.031355621
118.998871257
115.324955892
117.704593844
114.614916285
115.488937936
115.301181132
112.884677969
110.438264654
112.6502778
108.240899197
113.023166387
112.527688868
104.043703973
109.489717386
106.268769574
119.257297163
106.566003656
117.948329487
114.489725497
114.456525191
109.999122206
117.433950228
114.465193948
119.716223105
116.842383015
111.861664437
105.899876908
113.279206147
110.258116302
103.026957992
118.554341918
110.386592278
114.923608869
105.65660348
110.436806722
112.850456278
106.855534149
114.366720203
118.115810293
111.207449304
115.71452776
109.020851502
107.860424499
115.514346043
106.148005519
107.648776658
117.223154111
117.754933869
111.247577698
111.470321284
99.3738396753
122.50225163
112.524741035
112.51741425


[[[[0.040080098805495706, 0.02271163781671606],
   [-0.044345449318098404, 0.08359843599092803]],
  [[0.011541899891153326, 0.0721255214742231],
   [0.08306711716069212, -0.0009731244539543603]],
  [[-0.04372577668599995, -0.015881870855371516],
   [-0.07373257603118048, 0.020213502944061983]],
  [[-0.034931488538591945, 0.046454782451335844],
   [-0.04490072571086965, 0.0005602078827699311]]],
 [[-2.8986786751846556e-05, 2.4664632971173176e-05],
  [0.0027431169714636256, -0.0007468700991364072],
  [-0.0027415402223763697, -0.04281765490412362],
  [-0.3224607722812455, 0.32246077228124387]]]

In [63]:
[rew, reb] =[[[[-2.486813741889679, -2.4277744078594448],
   [2.4211824971485996, 2.4937044823235737]]],
 [[0.4345406279344458, -0.43454062793444576]]]

In [66]:
dnn ([2,-3] , rew, reb, 4, [])[0]

[[2, -3], [0.9970338663916688, 0.0029661336083310593]]

def back_prop(der_e_out, w, func, in_tensor):
    bw = [ta.mat_transp(k) for k in revers(w)] 
    rev_i = revers(in_tensor)
    if type(func) == int:
            func_list = [(func, rev_i[k]) for k in range(len(bw))]+[(0,0)]
    elif len(func) == len(bw):
            func_list =  [(revers(func)[k], rev_i[k]) for k in range(len(bw))]+[0]
    elif len(func) == len(bw)+1:
            func_list = [(revers(func)[k], rev_i[k]) for k in range(len(func))]
    else:
        return "wrong func"
    o_tensor=[]
    i_tensor=[der_e_out]
    for i in range(len(func_list)): 
        o_tensor.append(actvi_func(i_tensor[i], func_list[i]) )
        if i == len(bw):
            break
        i_tensor.append(ta.mat_dot(bw[i], o_tensor[i]))
    return revers(o_tensor)

def train_dnn_adp(data, k, width, func, der_func, step, softmax):
    w =ta.gene_random_tensor([1, len(data[0][0] ), width], (-1,1))+ ta.gene_random_tensor([k, width, width], (-1,1))+\
           ta.gene_random_tensor([1, width,  len(data[0][1] ),], (-1,1))
    b =ta.gene_random_tensor([k+1, width], (-1,1))+ ta.gene_random_tensor([1, len(data[0][1] )], (-1,1))
    w_grad_sum=ta.gene_blank_tensor([1, len(data[0][0] ), width] )+ta.gene_blank_tensor([k, width, width] )+\
                           ta.gene_blank_tensor([1, width,  len(data[0][1] )] )
    b_grad_sum=ta.gene_blank_tensor([k+1, width] )+ta.gene_blank_tensor( [1, len(data[0][1] )])
    while True:
        time =0
        for d in data:
            y = dnn(d[0] ,w, b, func, softmax)
            der_e_out =t.list_add(y[len(y)-1], t.list_nu( d[1],-1) )
            der_e_a=back_pro(der_e_out, y, w, der_func, softmax)
            der_e_b=der_e_a
            der_e_w=[ta.mat_dot(ta.colu_fi(y[i] ), [der_e_a[i] ])  for i in range(len(der_e_a))]
            w_grad_sum = t.list_power(t.list_add( t.list_power(w_grad_sum, 2) , t.list_power(der_e_w, 2) ), 1/2)  
            b_grad_sum = t.list_power(t.list_add( t.list_power(b_grad_sum, 2) , t.list_power(der_e_b, 2) ), 1/2) 
            w = t.list_add(w, t.list_muti( t.list_nu(der_e_w, -step) , t.list_power( zero_returrn1(w_grad_sum )  , -1) ))
            b = t.list_add(b, t.list_muti( t.list_nu(der_e_b, -step) , t.list_power( zero_returrn1(b_grad_sum )  , -1) ))
        if t.list_max(der_e_b+ der_e_w) <0.1:
            break
    return [w, b]      